In [6]:
import os
import sys
import time
import glob
import shutil
import numpy as np
import pandas as pd

import subprocess
from pathlib import Path
from subprocess import Popen, PIPE
import copy

from scipy import spatial
from scipy import ndimage
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy.table import Table
from astropy.stats import mad_std
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
# from astropy.stats import SigmaClip
from astropy.wcs.utils import skycoord_to_pixel
# from photutils.background import Background2D, MedianBackground,SExtractorBackground
from photutils.centroids import centroid_1dg, centroid_2dg
import warnings

from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

from photutils.segmentation import detect_sources, deblend_sources



In [16]:
def create_mask(id_in_list, data_img, wht_img):

    ### want to read in the cutout image and then make segmentation map from it. ###
    segment_map = detect_sources(data_img, threshold=1.5*wht_img, npixels=20)
    #print(segment_map.shape)


    segm_deblend = deblend_sources(data_img, segment_map,
                               npixels=20, nlevels=32, contrast=0.001,
                               progress_bar=True)
    
    mask_cutout = segm_deblend
    # wcs_seg = WCS(data_img[0].header)

    from skimage.draw import disk

    #ny, nx = mask_cutout.shape
    ny,nx = 101,101
    ic, jc = ny//2, nx//2
    label = mask_cutout.data[ic, jc]
 
    mask_cutout = mask_cutout.data

    mask = np.where(mask_cutout == int(label))

    mask_cutout[mask] = 0
    # mask_cutout[~mask] = 0


    return np.array(mask_cutout)


In [14]:
def make_stamp(id_, img, sig, ra, dec,psf_filename, seg=False):
    filepath = "/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/"

    folder = filepath + f"{str(id_)}_pysersic_files/"

    stamp_name = f"{str(id_)}_stamp.fits"
    mask_name = f"{str(id_)}_mask.fits"

    if os.path.exists(folder):
        print("Path already exists!")
        return
    else:
        os.makedirs(folder)

      ### for image stamp ###
        hdu=img
        wcs = WCS(hdu[0].header)
        size = (101, 101) #change to 60,60 for hst

        ### uncomment if need to use sky coords instead of pixel coords. ###
       
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, unit="deg")

        ## uncomment below for images which aren't in uJy. ###
        # old_ZP = -2.5*np.log10(hdu[0].header['PHOTFLAM']) - 5*np.log10(hdu[0].header['PHOTPLAM']) -2.408194
        # unit_to_uJy = 10**((23.9-old_ZP)/2.5)

        cutout = Cutout2D(hdu[0].data, coord, size, wcs =wcs)
        fits.writeto(os.path.join(folder, stamp_name), data=cutout.data, overwrite=True)#*unit_to_uJy add this back for hst
      
        ### for sigma map ###
        wcs_rms = WCS(img[0].header)
        rms_cutout = Cutout2D(sig[0].data, coord, size, wcs=wcs_rms).data
        rms_map = 1/np.sqrt(rms_cutout)
        fits.writeto(os.path.join(folder, str(id_)+"_rms.fits"), data = rms_map, overwrite = True)#*unit_to_uJy
        data_img = cutout.data
        wht_img = rms_map

        mask_cutout = create_mask(id_,data_img,wht_img)
        fits.writeto(os.path.join(folder, mask_name), data = mask_cutout, overwrite = True)

        shutil.copy(psf_filename, folder)

        return 0

In [17]:
id_ = 'rubies_uds_144195'

ra,dec = 34.32515592884,-5.143685286

f150_psf = '/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/f150w_psf_norm.fits'

wht_img_f150 = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f150w_v1.0_wht.fits')
sci_img_f150 = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f150w_v1.0_sci.fits')

make_stamp(id_, sci_img_f150, wht_img_f150,ra, dec, psf_filename=f150_psf, seg=True)


Deblending:   0%|          | 0/2 [00:00<?, ?it/s]

0